# SANS2D: I(Q) for sample and background

In this notebook, we will be reducing a sample and a background measurements to a one-dimensional $I(Q)$.

It assumes the detector data has been recorded in event mode, while the monitor data has been histogrammed.

The data used in this notebook has been published in [Manasi et al. (2021)](#References),
and we kindly thank the authors for allowing us to use their data.

**Outline:**

- We will begin by loading the data files containing the sample, direct, and background measurements.
- We will then apply some corrections to beamline components specific to the SANS2D beamline.
- This will be followed by some masking of some saturated or defect detector pixels
- Finally, the sample and background measurement will be converted to the $Q$ dimension

In [ ]:
import matplotlib.pyplot as plt
import scipp as sc
from ess import loki, sans
import scippneutron as scn

## Define reduction workflow parameters

We define here whether to include the effects of gravity,
as well as common time-of-flight, wavelength and $Q$ bins for all the measurements.

We also define a range of wavelengths for the monitors that are considered to not be part of the background.

In [ ]:
# Include effects of gravity?
gravity = True

tof_bins = sc.linspace(dim='tof', start=0, stop=100000, num=2, unit='us')

wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141,
                              unit='angstrom')

q_bins = sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom')

# Define the range of wavelengths for the monitors that are considered
# to not be part of the background
monitor_non_background_range = sc.array(dims=['wavelength'],
                                        values=[0.7, 17.1], unit='angstrom')

## Loading data files

We load the following files:

- The direct beam function for the main detector (gives detector efficiency as a function of wavelength)
- The sample measurement
- The direct measurement: this is the run with the empty sample holder/cuvette
- the background measurement: this is the run with only the solvent which the sample is placed in

In [ ]:
ds = sc.Dataset()

#Using only one-forth of the full spectra 245760 (reserved for first detector)
spectrum_size =  245760//4

direct_beam = loki.io.load_rkh_wav(
    loki.data.get_path('DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'))

ds['sample'] = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063114.nxs'),
                                   spectrum_size=spectrum_size, tof_bins=tof_bins)

ds['direct'] = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063091.nxs'),
                                   spectrum_size=spectrum_size, tof_bins=tof_bins)

ds['background'] = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063159.nxs'),
                                       spectrum_size=spectrum_size, tof_bins=tof_bins)
ds

## Apply corrections to pixel positions

We apply some corrections to the detector pixel and monitor positions,
as the geometry stored in the file is inaccurate.

In [ ]:
# Custom SANS2D position offsets
sample_pos_z_offset = 0.053 * sc.units.m
bench_pos_y_offset = 0.001 * sc.units.m
# There is some uncertainity here
monitor4_pos_z_offset = -6.719 * sc.units.m

# Geometry transformation
x_offset = -0.09288 * sc.units.m
y_offset = 0.08195 * sc.units.m

In [ ]:
ds.coords["pixel_width"] = 0.0035 * sc.units.m
ds.coords["pixel_height"] = 0.002033984375 * sc.units.m

# Change sample position
ds.coords["sample_position"].fields.z += sample_pos_z_offset
# Apply bench offset to pixel positions
ds.coords["position"].fields.y += bench_pos_y_offset

for key in ds:
    ds[key].attrs["monitor4"].value.coords["position"].fields.z += monitor4_pos_z_offset

# Now shift pixels positions to get the correct beam center
ds.coords['position'].fields.x += x_offset
ds.coords['position'].fields.y += y_offset

## Masking

The next step is to mask noisy and saturated pixels,
as well as a time-of-flight range that contains spurious artifacts from the beamline components.

**Note:** We use programatic masks here and not those stored in xml files.

### Mask bad pixels

We mask the edges of the detector, which are usually noisy.
We also mask the region close to the center of the beam,
so as to not include saturated pixels in our data reduction.

In [ ]:
mask_edges = (
    (sc.abs(ds.coords['position'].fields.x - x_offset) > sc.scalar(0.48, unit='m')) |
    (sc.abs(ds.coords['position'].fields.y - y_offset) > sc.scalar(0.45, unit='m')))

mask_center = sc.sqrt(
    ds.coords['position'].fields.x**2 +
    ds.coords['position'].fields.y**2) < sc.scalar(0.04, unit='m')

for key in ds:
    ds[key].masks['edges'] = mask_edges
    ds[key].masks['center'] = mask_center

We can inspect that the coordinate corrections and masking were applied correctly by opening the instrument view.

In [ ]:
scn.instrument_view(ds['sample'], pixel_size=0.0075)

### Mask Bragg peaks in time-of-flight

We will now take out the time regions with Bragg peaks from the beam stop and detector window,
although in reality the peaks appear only close to the beam stop,
and will make little difference to $I(Q)$.

This could be implemented as masking specific time bins for a specific region in space,
but for now we keep it simple.

In [ ]:
mask_tof_min = sc.scalar(13000.0, unit='us')
mask_tof_max = sc.scalar(15750.0, unit='us')
tof_masked_region = sc.concat([ds.coords['tof']['tof', 0],
                               mask_tof_min, mask_tof_max,
                               ds.coords['tof']['tof', -1]], dim='tof')

binned = sc.Dataset()
for key in ds:
    binned[key] = sc.bin(ds[key], edges=[tof_masked_region])
    binned[key].masks['bragg_peaks'] = sc.array(dims=['tof'], values=[False, True, False])
binned

## Use to_I_of_Q workflow

We now reduce the sample and the background measurements to `Q` using the `sans.to_I_of_Q` workflow.

In that process,
the intensity as a function of `Q` is normalized using the direct measurement and direct beam function.

The workflow needs monitor data from the sample, background, and direct runs to compute the normalization.
It accepts those in the form of a dictionaries:

In [ ]:
sample_monitors = {'incident': binned['sample'].attrs["monitor2"].value,
                   'transmission': binned['sample'].attrs["monitor4"].value}

direct_monitors = {'incident': binned['direct'].attrs["monitor2"].value,
                   'transmission': binned['direct'].attrs["monitor4"].value}

background_monitors = {'incident': binned['background'].attrs["monitor2"].value,
                       'transmission': binned['background'].attrs["monitor4"].value}

We then call the workflow on the sample and direct runs:

In [ ]:
sample_q = sans.to_I_of_Q(data=binned['sample'],
    data_monitors=sample_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    monitor_non_background_range=monitor_non_background_range)
sample_q.plot()

In [ ]:
background_q = sans.to_I_of_Q(data=binned['background'],
    data_monitors=background_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    monitor_non_background_range=monitor_non_background_range)
background_q.plot()

We are now in a position to subtract the background from the sample measurement:

In [ ]:
result = sample_q.bins.sum() - background_q.bins.sum()
result

In [ ]:
fig1, ax1 = plt.subplots(1, 2, figsize=(10, 4))
sc.plot(result, ax=ax1[0])
sc.plot(result, norm='log', ax=ax1[1])
fig1

<div class="alert alert-info">

**Note**

Instead of `.bins.sum()`,
one could use `sc.histogram()` above to define different `Q` bins compared to the ones defined at the top of the notebook.
This can be done in event mode, see [here](https://scipp.github.io/user-guide/binned-data/computation.html#Subtraction).

There may be performance advantages to first use a coarse `Q` binning when the computing `I(Q)` numerator,
and use finer binning for the final results.

<div>

## Wavelength bands

It is often useful to process the data in a small number (~10) of separate wavelength bands.

This can be achieved by requesting 10 bands from the `to_I_of_Q` workflow via the `wavelength_bands` argument.

In [ ]:
wavelength_bands = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=11,
                               unit='angstrom')

sample_slices = sans.to_I_of_Q(data=binned['sample'],
    data_monitors=sample_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_bands=wavelength_bands,
    monitor_non_background_range=monitor_non_background_range)

background_slices = sans.to_I_of_Q(data=binned['background'],
    data_monitors=background_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_bands=wavelength_bands,
    monitor_non_background_range=monitor_non_background_range)

result_slices = sample_slices.bins.sum() - background_slices.bins.sum()
result_slices

In [ ]:
collapsed = sc.collapse(result_slices, keep='Q')

fig2, ax2 = plt.subplots(1, 2, figsize=(10, 4))
sc.plot(collapsed, ax=ax2[0])
sc.plot(collapsed, norm='log', legend=False, ax=ax2[1])
fig2

## References

- Manasi I., Andalibi M. R., Atri R. S., Hooton J., King S. M., Edler K. J., **2021**, *Self-assembly
of ionic and non-ionic surfactants in type IV cerium nitrate and urea based deep eutectic solvent*,
[J. Chem. Phys. 155, 084902](https://doi.org/10.1063/5.0059238)